In [131]:
import warnings
warnings.filterwarnings("ignore")

from typing import List, Tuple
from helper.helper_functions import load_dataset, save_model, get_features_and_target, encode_all_features
from helper.fairness_functions import statistical_measures
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeClassifier

In [132]:
data = load_dataset('../data/assignment2_income_cleaned.xlsx')

In [133]:
# Splitting the data into features (X) and target (y)
X, y = get_features_and_target(data, 'income')
columns_to_exclude = ['age', 'ability to speak english', 'gave birth this year']
# Encoding the features and target, and excluding some columns
X_encoded, y_encoded = encode_all_features(X, y, [])
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

In [134]:
X_train

,age,education,workinghours,ability to speak english,workclass_governmental,workclass_no paid work,workclass_private,workclass_self employed,marital status_Divorced,marital status_Husband,...,occupation_Management/Business,occupation_Military Services,occupation_Office/Administrative Support,occupation_Production/Assembly,occupation_Protective Services,occupation_Repair/Maintenance,occupation_Sales,"occupation_Science, Engineering, Technology",occupation_Service/Hospitality,occupation_Transport
6317,22,16,36,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
740,61,22,40,1,0,0,0,1,1,0,...,0,0,0,0,0,0,1,0,0,0
3781,48,16,40,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
7850,62,18,65,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2963,53,19,44,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,22,19,25,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5191,24,16,28,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5390,35,16,40,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
860,23,20,40,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [135]:
X_test['sex'] = X_test['sex_Male'] * 1
X_train['sex'] = X_train['sex_Male'] * 1
X_test = X_test.drop(columns=['sex_Male', 'sex_Female'])
X_train = X_train.drop(columns=['sex_Male', 'sex_Female'])

DI, DS, EO, EOdds = statistical_measures(X_train, y_train, X_test, y_test, 'sex', use_lib_implementation=False)
    
print(f"Disparate Impact (DI): {DI:.3f}")
print(f"Discrimination Score (DS): {DS:.3f}")
print(f"Equal Opportunity Difference (EO): {EO:.3f}")
print(f"Equalized Odds (EOdds): {EOdds:.3f}")

Disparate Impact (DI): 0.525
Discrimination Score (DS): -0.198
Equal Opportunity Difference (EO): 0.179
Equalized Odds (EOdds): 0.097


In [136]:
from sklearn.metrics import precision_score, recall_score, f1_score

def get_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Tuple[float, float, float, float]:
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return accuracy, precision, recall, f1

def split_male_female_metrics(data: pd.DataFrame, is_encoded: bool = True) -> pd.DataFrame:
    
    X_, y_ = get_features_and_target(data, 'income')
    if not is_encoded:
        X_, y_ = encode_all_features(X_, y_, ['sex'])
        X_['sex'] = X_['sex'].map({'Female': 0, 'Male': 1})
        
    model = DecisionTreeClassifier(random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)
        
    X_male = X_[X_['sex'] == 1]
    X_female = X_[X_['sex'] == 0]
    y_male = y_[X_['sex'] == 1]
    y_female = y_[X_['sex'] == 0]
    
    print(f"Length of Male dataset: {len(X_male)}")
    print(f"Length of Female dataset: {len(X_female)}")
    
    # X_male_train, X_male_test, y_male_train, y_male_test = train_test_split(X_male, y_male, test_size=0.2, random_state=42)
    X_male_train = X_train[X_train['sex'] == 1]
    X_male_test = X_test[X_test['sex'] == 1]
    y_male_train = y_train[X_train['sex'] == 1]
    y_male_test = y_test[X_test['sex'] == 1]
    
    clf_male = model # DecisionTreeClassifier(random_state=42)
    # clf_male.fit(X_male_train, y_male_train)
    y_male_pred = clf_male.predict(X_male_test)
    
    male_accuracy, male_precision, male_recall, male_f1 = get_metrics(y_male_test, y_male_pred)
    
    # X_female_train, X_female_test, y_female_train, y_female_test = train_test_split(X_female, y_female, test_size=0.2, random_state=42)
    X_female_train = X_train[X_train['sex'] == 0]
    X_female_test = X_test[X_test['sex'] == 0]
    y_female_train = y_train[X_train['sex'] == 0]
    y_female_test = y_test[X_test['sex'] == 0]
    
    clf_female = model # DecisionTreeClassifier(random_state=42)
    # clf_female.fit(X_female_train, y_female_train)
    y_female_pred = clf_female.predict(X_female_test)
    
    female_accuracy, female_precision, female_recall, female_f1 = get_metrics(y_female_test, y_female_pred)
    
    # Compare the model's accuracy for male and female groups
    metrics_table = pd.DataFrame({
        'Gender': ['Male', 'Female'],
        'Accuracy': [male_accuracy, female_accuracy],
        'Precision': [male_precision, female_precision],
        'Recall': [male_recall, female_recall],
        'F1-score': [male_f1, female_f1]
    })
    
    return metrics_table

In [137]:
metrics_table = split_male_female_metrics(data, is_encoded=False)
metrics_table

Length of Male dataset: 6000
Length of Female dataset: 3000


,Gender,Accuracy,Precision,Recall,F1-score
0,Male,0.694536,0.637131,0.60521,0.620761
1,Female,0.766892,0.458333,0.52381,0.488889


In [138]:
from imblearn.over_sampling import ADASYN

adasyn = ADASYN(random_state=42)

X_, y_ = get_features_and_target(data, 'income')
X_, y_ = encode_all_features(X_, y_, ['sex'])
X_['sex'] =  X_['sex'].map({'Female': 0, 'Male': 1})

X_with_y = pd.concat([X_, y_], axis=1)

X_ = X_with_y.drop(columns=['sex'])
y_ = X_with_y['sex']

# Resample the dataset
X_resampled, y_resampled = adasyn.fit_resample(X_, y_)

X_with_y_resampled = pd.concat([X_resampled, y_resampled], axis=1)

In [139]:
X_with_y_resampled['sex'].value_counts() / len(X_with_y_resampled) * 100

sex
1    52.232959
0    47.767041
Name: count, dtype: float64

In [140]:
counts = X_with_y_resampled.groupby(['sex', 'income']).size().unstack(fill_value=0)
total_counts = X_with_y_resampled['sex'].value_counts()
# Calculate the percentage of each income level for each sex
percentage = counts.div(total_counts, axis=0) * 100

In [141]:
percentage

income,0,1
sex,,
0,84.344815,15.655185
1,58.900000,41.100000


In [142]:
X_, y_ = get_features_and_target(X_with_y_resampled, 'income')

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=42)

DI, DS, EO, EOdds = statistical_measures(X_train, y_train, X_test, y_test, 'sex', use_lib_implementation=False)
    
print(f"Disparate Impact (DI): {DI:.3f}")
print(f"Discrimination Score (DS): {DS:.3f}")
print(f"Equal Opportunity Difference (EO): {EO:.3f}")
print(f"Equalized Odds (EOdds): {EOdds:.3f}")

Disparate Impact (DI): 0.355
Discrimination Score (DS): -0.252
Equal Opportunity Difference (EO): 0.094
Equalized Odds (EOdds): 0.161


In [144]:
metrics_table = split_male_female_metrics(X_with_y_resampled, is_encoded=True)

Length of Male dataset: 6000
Length of Female dataset: 5487


In [145]:
metrics_table

,Gender,Accuracy,Precision,Recall,F1-score
0,Male,0.684383,0.614786,0.626984,0.620825
1,Female,0.845581,0.488889,0.543210,0.514620
